In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [45]:
# Filter DataFrame for total_votes >= 20
df_tv = df.filter(df.total_votes >= 20)
df_tv.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   31044774|R26CC1STPSUHBS|B01019T6O0|     510976915|Microsoft Windows...|        Software|          1|           27|         39|   N|                N|Do NOT take the "...|Do NOT take the &...| 2015-08-31|
|         US|   51362864| R7K83D59T9KWS|B00ZSI7Y3U|     604043733|Microsoft Windows...|        Software|          1|    

In [46]:
# Filter DataFrame for helpful_votes >= 50%
df_hv = df.filter((df.helpful_votes / df.total_votes) >= 0.5)
df_hv.show(5)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   20193077|R1XU1B93402SYJ|B00N4OLCRO|     776572654|Photoshop Element...|        Software|          1|    

In [47]:
# Filter to a vine DataFrame (paid and unpaid)
df_vine = df.filter(df.vine == "Y") # Paid reviews
df_vine_N = df.filter(df.vine == "N") # Unpaid reviews
df_vine.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47098142|R2S99I8HFU0KX1|B00V8XHZK6|     618336003|Adobe Acrobat Pro...|        Software|          2|            4|          7|   Y|                N|It's not very goo...|Good lord. Acroba...| 2015-08-31|
|         US|   52079633|R25WYQUWVV9B76|B00V8XHZK6|     618336003|Adobe Acrobat Pro...|        Software|          5|    

In [58]:
# Create df for paid five star reviews
df_vine_five = df_vine.filter(df_vine.star_rating == 5)

In [59]:
# Create df for unpaid five star reviews
df_vine_N_five = df_vine_N.filter(df_vine_N.star_rating == 5)

In [57]:
# Show results for analysis
total_reviews = df.select(["star_rating"]).count()
total_paid_reviews = df_vine.count()
total_unpaid_reviews = df_vine_N.count()
total_paid_5_star_reviews = df_vine_five.count()
total_unpaid_5_star_reviews = df_vine_N_five.count()

print("Total Reviews: %s" % total_reviews)
print("Total Paid Reviews: %s" % total_paid_reviews)
print("Total Unpaid Reviews: %s" % total_unpaid_reviews)
print("Total Paid 5 Star Reviews: %s" % total_paid_5_star_reviews)
print("Total Unpaid 5 Star Reviews: %s" % total_unpaid_5_star_reviews)

Total Reviews: 341931
Total Paid Reviews: 10415
Total Unpaid Reviews: 331516
Total Paid 5 Star Reviews: 3381
Total Unpaid 5 Star Reviews: 150566
